In [5]:
import config
import query_helper
import json
import requests
from bs4 import BeautifulSoup
api_key = config.API_key
import time 
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

# Below 2 cells creates database

In [2]:
db_name = 'colleges'
## Connect to DB server on AWS
## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "colleges"       #can say what database we expect everything loaded into
)
cursor = cnx.cursor()

In [ ]:
# def create_database(cursor, database):
#     try:
#         cursor.execute(
#             "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
#     except mysql.connector.Error as err:
#         print("Failed creating database: {}".format(err))
#         exit(1)

# try:
#     cursor.execute("USE {}".format(db_name))
# except mysql.connector.Error as err:
#     print("Database {} does not exists.".format(db_name))
#     if err.errno == errorcode.ER_BAD_DB_ERROR:
#         create_database(cursor, db_name)
#         print("Database {} created successfully.".format(db_name))
#         cnx.database = db_name
#     else:
#         print(err)
#         exit(1)

In [3]:
#write a query entry to creat a table
payscale_colleges = """
CREATE TABLE payscale_colleges (
      uni varchar(100) PRIMARY KEY,
      uni_type varchar(100),
      early_car_pay int,
      mid_car_pay int,
      high_meaning float,
      stem_deg float
    );
"""

In [4]:
query_helper.create_table(payscale_colleges)

Creating a new table
OK


In [6]:
page = requests.get("https://www.payscale.com/college-salary-report/bachelors")
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
seven_day = soup.find(id="detailed-forecast-body")

#4
forecast_items = seven_day.find_all(class_="col-sm-10 forecast-text")

#5
tonight = forecast_items
for i in tonight:
    print(i.get_text())

In [89]:
soup_1 = soup.find('tbody')
# soup_1
soup_uni_names = soup_1.find_all(class_="datatable-logo__text")
# soup_uni_names
uni_names = [uni.get_text() for uni in soup_uni_names]
uni_names

['Harvey Mudd College',
 'Massachusetts Institute of Technology',
 'Samuel Merritt University',
 'United States Naval Academy',
 'California Institute of Technology',
 'Harvard University',
 'Stanford University',
 'Albany College of Pharmacy and Health Sciences',
 'United States Military Academy',
 'Webb Institute',
 'United States Merchant Marine Academy',
 'SUNY Maritime College',
 'Stevens Institute of Technology',
 'Colorado School of Mines',
 'Princeton University',
 'Yale University',
 'United States Air Force Academy',
 'Carnegie Mellon University',
 'Rose-Hulman Institute of Technology',
 'Charles R Drew University of Medicine and Science',
 'Worcester Polytechnic Institute',
 'Santa Clara University',
 'Lehigh University',
 'Rensselaer Polytechnic Institute',
 'University of Pennsylvania']

In [90]:
soup_uni_type = soup_1.find_all(class_="pxl-hidden-sm-down text-center")
uni_type = [i.get_text() for i in soup_uni_type]
uni_type


['1',
 'Engineering, Liberal Arts School, Private School',
 '$88,800',
 '$158,200',
 '55%',
 '85%',
 '2',
 'Engineering, Private School, Research University',
 '$86,300',
 '$155,200',
 '52%',
 '69%',
 '3',
 'Private School',
 '$91,200',
 '$154,100',
 '90%',
 '0%',
 '4',
 'Engineering, Liberal Arts School, Sober School, For Sports Fans, State School',
 '$80,100',
 '$152,800',
 '64%',
 '58%',
 '5',
 'Engineering, Private School, Research University',
 '$84,100',
 '$151,600',
 '53%',
 '97%',
 '6',
 'Ivy League, Private School, Research University, For Sports Fans',
 '$74,800',
 '$146,800',
 '54%',
 '19%',
 '7',
 'Engineering, Private School, Research University, For Sports Fans',
 '$79,000',
 '$145,200',
 '56%',
 '51%',
 '8',
 'Private School',
 '$81,000',
 '$144,800',
 '82%',
 '2%',
 '9',
 'Liberal Arts School, Sober School, For Sports Fans, State School',
 '$80,200',
 '$144,000',
 '61%',
 '39%',
 '10',
 'Engineering, Private School',
 '$80,800',
 '$141,800',
 'N/A',
 '100%',
 '11',
 'En

In [91]:
# x = int((uni_type[2].strip('$').replace(',','')))
# # y =x.replace(',', '')
# # int(y)
# x     
# # int(x)

In [48]:
list_of_dict = []
temp_dict_ ={}
for i in uni_names:
    temp_dict_['university']=i
    list_of_dict.append(temp_dict_.copy())  #need to 


In [49]:
list_of_dict

[{'university': 'Harvey Mudd College'},
 {'university': 'Massachusetts Institute of Technology'},
 {'university': 'Samuel Merritt University'},
 {'university': 'United States Naval Academy'},
 {'university': 'California Institute of Technology'},
 {'university': 'Harvard University'},
 {'university': 'Stanford University'},
 {'university': 'Albany College of Pharmacy and Health Sciences'},
 {'university': 'United States Military Academy'},
 {'university': 'Webb Institute'},
 {'university': 'United States Merchant Marine Academy'},
 {'university': 'SUNY Maritime College'},
 {'university': 'Stevens Institute of Technology'},
 {'university': 'Colorado School of Mines'},
 {'university': 'Princeton University'},
 {'university': 'Yale University'},
 {'university': 'United States Air Force Academy'},
 {'university': 'Carnegie Mellon University'},
 {'university': 'Rose-Hulman Institute of Technology'},
 {'university': 'Charles R Drew University of Medicine and Science'},
 {'university': 'Worce

In [47]:
list_of_dict[1]

{'University': 'Massachusetts Institute of Technology'}

## Below block puts our items into list of dicts 

In [87]:
i = -1 #use this to control iteration thru list_of_dict
for counter, value in enumerate(uni_type):
    if counter%6==0:
        i+=1 #iterate to next college in list
        continue
    elif(counter%6==1):
        list_of_dict[i]['uni_type']=value
    elif(counter%6==2):
        list_of_dict[i]['early_car_pay']= (int((value.strip('$').replace(',',''))))
    elif(counter%6==3):
        list_of_dict[i]['med_car_pay']=(int((value.strip('$').replace(',',''))))
    elif(counter%6==4):
        if value =='N/A':
            list_of_dict[i]['high_meaning']= 0
        else:
            list_of_dict[i]['high_meaning']= float(value.strip('%'))
    else:
        list_of_dict[i]['stem_deg']= float(value.strip('%'))
        

In [88]:
list_of_dict

[{'university': 'Harvey Mudd College',
  'uni_type': 'Engineering, Liberal Arts School, Private School',
  'early_car_pay': 88800,
  'med_car_pay': 158200,
  'high_meaning': 55.0,
  'stem_deg': 85.0},
 {'university': 'Massachusetts Institute of Technology',
  'uni_type': 'Engineering, Private School, Research University',
  'early_car_pay': 86300,
  'med_car_pay': 155200,
  'high_meaning': 52.0,
  'stem_deg': 69.0},
 {'university': 'Samuel Merritt University',
  'uni_type': 'Private School',
  'early_car_pay': 91200,
  'med_car_pay': 154100,
  'high_meaning': 90.0,
  'stem_deg': 0.0},
 {'university': 'United States Naval Academy',
  'uni_type': 'Engineering, Liberal Arts School, Sober School, For Sports Fans, State School',
  'early_car_pay': 80100,
  'med_car_pay': 152800,
  'high_meaning': 64.0,
  'stem_deg': 58.0},
 {'university': 'California Institute of Technology',
  'uni_type': 'Engineering, Private School, Research University',
  'early_car_pay': 84100,
  'med_car_pay': 151600,